In [ ]:
# Stratified Sampling

In [48]:
import sys
import os
from pathlib import Path
import importlib
sys.path.append('..')

import pandas as pd

import data.dataframe_preparation as preparation
from data.custom_widgets import ReportsLabeler
import data
importlib.reload(data.custom_widgets)

############### CONFIG ###############
FIRM_METADATA = os.path.abspath("../input_files/Firm_Metadata.csv")
DATA_INPUT_PATH = os.path.abspath("../input_files/annual_reports/")
MASTER_DATA_PATH = os.path.abspath("../input_files/annual_reports/Firm_AnnualReport_DF.csv")
LABEL_OUTPUT_FN = 'Firm_AnnualReport_Labels_DF.pkl'

HOLD_OUT_YEAR = 2019
HOLD_OUT_COMPANY = 'gb_unilever_plc'

SEED = 2
######################################

# Create master output file if not exits
master_file = Path(MASTER_DATA_PATH)
if not master_file.is_file():
    df = preparation.get_df(input_path=DATA_INPUT_PATH, report_type_mappings={"20F": "AR"}, selected_report_types={"AR"}, include_text=False, include_page_no=False, include_toc=False)
    df = df.set_index("id")
    # Add additional labelling columns
    df['should_label'] = False
    df['is_labelled'] = False
    
    # Load the meta data
    df_meta = pd.read_csv(FIRM_METADATA)
    df_meta = df_meta.set_index('id')
    df['company_id'] = df['country'] + "_" + df['company']
    df = df.drop(columns=['country'])
    df = df.merge(df_meta, left_on='company_id', right_index=True)
    
    # TODO: Raise/Alert or remove if all outputs are there!
    df = df[df['output_file'].notna()]
    
    # Select all reports from year and company
    df_to_label = df[(df.year == HOLD_OUT_YEAR) | (df.company_id == HOLD_OUT_COMPANY)].copy()
    df_to_label['should_label'] = True
    nr_held_out_reports = len(df_to_label)
    df.update(df_to_label)
    
    # Remaining reports: Take sample with stratification along variables. n = amount of reports in each year AND industry...
    df_to_label = df[df.should_label == False] #.sample(TOTAL_SAMPLE_SIZE - nr_held_out_reports, random_state=SEED)
    df_to_label = df_to_label.groupby(['year', 'icb_industry']).apply(lambda x: x.sample(n=1, random_state=SEED))
    df_to_label = df_to_label.reset_index(2).reset_index(drop=True).set_index("id")
    
    df_to_label['should_label'] = True
    df.update(df_to_label)
    df.to_csv(MASTER_DATA_PATH)
    

Output()

Output()

Output()